# Example of using SpecialtyInsurance Simulator


In [1]:
### 1. Set Simulation Parameters

import os
from logger.arguments import get_arguments

# Read arguments from logger.arguments
sim_args, manager_args, broker_args, syndicate_args, reinsurancefirm_args, shareholder_args, risk_args, seed = get_arguments()

# Reset arguments
sim_args["max_time"] = 30   # Simulation time span unit day
manager_args["lead_top_k"] = 3   # Number of syndicates competing for the lead quote
manager_args["follow_top_k"] = 2   # Number of syndicates following the lead strategy
broker_args["num_brokers"] = 1   # Number of brokers in the insurance market
syndicate_args["num_syndicates"] = 3   # Number of syndicates in the insurance market
shareholder_args["num_shareholders"] = 1   # Number of shareholders in the insurance market
risk_args["num_risks"] = 31  # Number of risks
risk_args["num_categories"] = 4  # Number of risk categories

# No reinsurance mechanism included in this stage
with_reinsurance = False   

# Nomber of risk models loaded to all syndicates
num_risk_models = 1   


In [2]:
### 2. Generate Catastrophes

from environment.risk_generator import RiskGenerator

# Create catastrophe list and catastrophe configurations
catastrophes, broker_risks, fair_market_premium, risk_model_configs = RiskGenerator(num_risk_models, sim_args, risk_args, seed).generate_risks()
print(catastrophes)
print(broker_risks)
#print("catastrophe starts at time", catastrophes[0].get("catastrophe_start_time"))
#print("catastrophe category is", catastrophes[0].get("catastrophe_category"), "catastrophe_id is", catastrophes[0].get("catastrophe_id"))


[{'catastrophe_id': 0, 'catastrophe_start_time': 26, 'catastrophe_category': 1, 'catastrophe_value': array([0.32075186])}, {'catastrophe_id': 1, 'catastrophe_start_time': 8, 'catastrophe_category': 2, 'catastrophe_value': array([0.51470285])}, {'catastrophe_id': 2, 'catastrophe_start_time': 9, 'catastrophe_category': 3, 'catastrophe_value': array([0.25467457])}, {'catastrophe_id': 3, 'catastrophe_start_time': 15, 'catastrophe_category': 3, 'catastrophe_value': array([0.26632473])}]
[{'risk_id': 0, 'risk_start_time': 0, 'risk_factor': 0.5157968626301841, 'risk_category': 3, 'risk_value': 1000.0}, {'risk_id': 1, 'risk_start_time': 1, 'risk_factor': 0.42527290354159575, 'risk_category': 3, 'risk_value': 1000.0}, {'risk_id': 2, 'risk_start_time': 2, 'risk_factor': 0.47069174073820397, 'risk_category': 1, 'risk_value': 1000.0}, {'risk_id': 3, 'risk_start_time': 3, 'risk_factor': 0.4092414922116555, 'risk_category': 0, 'risk_value': 1000.0}, {'risk_id': 4, 'risk_start_time': 4, 'risk_factor'

In [3]:
### 3. Generate Insurance Market

from environment.market_generator import MarketGenerator

# Create lists of brokers, syndicates, reinsurancefirms, and shareholders
brokers, syndicates, reinsurancefirms, shareholders = MarketGenerator(with_reinsurance, 
                                                                      num_risk_models, 
                                                                      sim_args, 
                                                                      broker_args, 
                                                                      syndicate_args, 
                                                                      reinsurancefirm_args, 
                                                                      shareholder_args, 
                                                                      risk_model_configs).generate_agents()
for broker_id in range(len(brokers)):
    print(brokers[broker_id].data())
for syndicate_id in range(len(syndicates)):
    print(syndicates[syndicate_id].data())
    

{'broker_id': '0', 'broker_risk': [], 'broker_quote': []}
{'syndicate_id': '0', 'initial_capital': 10000000, 'current_capital': 10000000, 'premium_internal_weight': 0.5, 'interest_rate': 0.001, 'play_leader_in_contracts': [], 'play_follower_in_contracts': [], 'loss_experiency_weight': 0.2, 'volatility_weight': 0, 'underwriter_markup_recency_weight': 0.2, 'upper_premium_limit': 1.2, 'lower_premium_limit': 0.85, 'premium_reserve_ratio': 0.5, 'minimum_capital_reserve_ratio': 1, 'maximum_scaling_factor': 1, 'market_entry_probability': 0.3, 'exit_capital_threshold': 0.6, 'exit_time_limit': 24, 'premium_sensitivity': 5, 'acceptance_threshold_friction': 0.9}
{'syndicate_id': '1', 'initial_capital': 10000000, 'current_capital': 10000000, 'premium_internal_weight': 0.5, 'interest_rate': 0.001, 'play_leader_in_contracts': [], 'play_follower_in_contracts': [], 'loss_experiency_weight': 0.2, 'volatility_weight': 0, 'underwriter_markup_recency_weight': 0.2, 'upper_premium_limit': 1.2, 'lower_premiu

In [4]:
### 4. Input risk from broker

from environment.event_generator import EventGenerator

current_time = 0
broker_risk_events = EventGenerator(risk_model_configs).generate_risk_events(sim_args, brokers, broker_risks)
catastrophe_events = EventGenerator(risk_model_configs).generate_catastrophe_events(catastrophes)
attritional_loss_events = EventGenerator(risk_model_configs).generate_attritional_loss_events(sim_args, broker_risks)
broker_premium_events = EventGenerator(risk_model_configs).generate_premium_events(sim_args)
broker_claim_events = EventGenerator(risk_model_configs).generate_claim_events(sim_args)

for i in range(len(broker_risk_events)):
    print("risk_id:", broker_risk_events[i].risk_id, "broker_id:", broker_risk_events[i].broker_id, "risk_start_time:", broker_risk_events[i].risk_start_time,
         "risk_end_time:", broker_risk_events[i].risk_end_time, "risk_factor:", broker_risk_events[i].risk_factor,
         "risk_category:", broker_risk_events[i].risk_category, "risk_value:", broker_risk_events[i].risk_value)


risk_id: 0 broker_id: 0 risk_start_time: 0 risk_end_time: 1 risk_factor: 0.5157968626301841 risk_category: 3 risk_value: 1000.0
risk_id: 1 broker_id: 0 risk_start_time: 1 risk_end_time: 2 risk_factor: 0.42527290354159575 risk_category: 3 risk_value: 1000.0
risk_id: 2 broker_id: 0 risk_start_time: 2 risk_end_time: 3 risk_factor: 0.47069174073820397 risk_category: 1 risk_value: 1000.0
risk_id: 3 broker_id: 0 risk_start_time: 3 risk_end_time: 4 risk_factor: 0.4092414922116555 risk_category: 0 risk_value: 1000.0
risk_id: 4 broker_id: 0 risk_start_time: 4 risk_end_time: 5 risk_factor: 0.5308107023198277 risk_category: 3 risk_value: 1000.0
risk_id: 5 broker_id: 0 risk_start_time: 5 risk_end_time: 6 risk_factor: 0.4893654282771117 risk_category: 3 risk_value: 1000.0
risk_id: 6 broker_id: 0 risk_start_time: 6 risk_end_time: 7 risk_factor: 0.4898672207866545 risk_category: 0 risk_value: 1000.0
risk_id: 7 broker_id: 0 risk_start_time: 7 risk_end_time: 8 risk_factor: 0.40476982047218 risk_categor

In [5]:
### 5. Create Multi-agent Environment to get access to the market performance

import gym
import numpy as np
from environment.environment import SpecialtyInsuranceMarketEnv
from environment.event_generator import EventGenerator
from manager.ai_model.action import Action
from manager import EventHandler, MarketManager
from logger import logger

class MultiAgentBasedModel(SpecialtyInsuranceMarketEnv):

    def __init__(self, sim_args, manager_args, broker_args, syndicate_args, reinsurancefirm_args, shareholder_args, risk_args, 
                 brokers, syndicates, reinsurancefirms, shareholders, catastrophes, broker_risks, fair_market_premium,
                 risk_model_configs, with_reinsurance, num_risk_models, dt = 1):

        self.sim_args = sim_args
        self.maxstep = self.sim_args["max_time"]
        self.manager_args = manager_args
        self.broker_args = broker_args
        self.syndicate_args = syndicate_args
        self.reinsurancefirm_args = reinsurancefirm_args
        self.shareholder_args = shareholder_args
        self.risk_args = risk_args
        self.brokers = brokers
        self.syndicates = syndicates
        self.reinsurancefirms = reinsurancefirms
        self.shareholders = shareholders
        self.catastrophes = catastrophes
        self.broker_risks = broker_risks
        self.fair_market_premium =fair_market_premium
        self.initial_catastrophes = catastrophes
        self.risk_model_configs = risk_model_configs
        self.with_reinsurance = with_reinsurance
        self.num_risk_models = num_risk_models
        self.dt = dt
        self.mm = None
        self.event_handler = None

        # Active syndicate list
        self.syndicate_active_list = []
        # Initialise events, actions, and states 
        self.attritional_loss_events = []
        self.catastrophe_events = []
        self.broker_risk_events = []
        self.broker_premium_events = []
        self.broker_claim_events = []
        self.action_map_dict = {}
        self.state_encoder_dict = {}

        # Define Action Space, Define Observation Space
        self.n = len(self.syndicates)
        self.agents = {self.syndicates[i].syndicate_id for i in range(self.n)} 
        self._agent_ids = set(self.agents)
        self.dones = set()
        self._spaces_in_preferred_format = True
        self.observation_space = gym.spaces.Dict({
            self.syndicates[i].syndicate_id: gym.spaces.Box(low=np.array([-10000000,-10000000,-30000000,-30000000,-30000000,-30000000]), 
                                                     high=np.array([10000000,10000000,30000000,30000000,30000000,30000000]), dtype = np.float32) for i in range(self.n)
        })
        self.action_space = gym.spaces.Dict({
            self.syndicates[i].syndicate_id: gym.spaces.Box(0.5, 0.9, dtype = np.float32) for i in range(self.n)})

        super(MultiAgentBasedModel, self).__init__(sim_args = self.sim_args, 
                                                   manager_args = self.manager_args , 
                                                   brokers = self.brokers, 
                                                   syndicates = self.syndicates, 
                                                   reinsurancefirms = self.reinsurancefirms, 
                                                   shareholders = self.shareholders, 
                                                   catastrophes = self.catastrophes, 
                                                   broker_risks = self.broker_risks,
                                                   fair_market_premium = self.fair_market_premium,
                                                   risk_model_configs = self.risk_model_configs, 
                                                   with_reinsurance = self.with_reinsurance, 
                                                   num_risk_models = self.num_risk_models,
                                                   dt = 1)
        
        # Log data
        self.cumulative_bankruptcies = 0
        self.cumulative_market_exits = 0
        self.cumulative_unrecovered_claims = 0.0
        self.cumulative_claims = 0.0
        self.logger = logger.Logger(self.num_risk_models, self.catastrophes, self.brokers, self.syndicates)

        # Reset the environmnet
        self.reset()

    def reset(self, seed = None, options = None):
        super().reset(seed = seed)
        
        # Broker risk event daily: TODO: broker generate risk according to poisson distribution
        # Catastrophe event 
        self.catastrophe_events = catastrophe_events
        # Attritioal loss event daily
        self.attritional_loss_events = attritional_loss_events
        # Broker risk event daily: TODO: broker generate risk according to poisson distribution
        self.broker_risk_events = broker_risk_events
        # Broker pay premium according to underwritten contracts
        self.broker_premium_events = broker_premium_events
        # Broker ask for claim if the contract reaches the end time
        self.broker_claim_events = broker_claim_events
        # Initiate event handler
        self.event_handler = EventHandler(self.maxstep, self.catastrophe_events, self.attritional_loss_events, self.broker_risk_events, self.broker_premium_events, self.broker_claim_events)
        # Initiate market manager
        self.mm = MarketManager(self.maxstep, self.manager_args, self.brokers, self.syndicates, self.reinsurancefirms, self.shareholders, self.catastrophes, 
                                self.fair_market_premium,
                                self.risk_model_configs, self.with_reinsurance, self.num_risk_models, self.catastrophe_events, self.attritional_loss_events, 
                                self.broker_risk_events, self.broker_premium_events, self.broker_claim_events, self.event_handler)
        self.mm.evolve(self.dt)
        
        # Set per syndicate active status and build status list
        self.syndicate_active_list = []   # Store syndicates currently in the market
        for sy in range(len(self.mm.market.syndicates)):
            if self.mm.market.syndicates[sy].status == True:
                self.syndicate_active_list.append(self.mm.market.syndicates[sy].syndicate_id)

        # Create action map and state list
        info_dict = {}
        for sy in range(len(self.mm.market.syndicates)):
            self.action_map_dict[self.mm.market.syndicates[sy].syndicate_id] = self.action_map_creator(self.mm.market.syndicates[sy], 0)
            self.state_encoder_dict[self.mm.market.syndicates[sy].syndicate_id] = self.state_encoder(self.mm.market.syndicates[sy].syndicate_id)
            info_dict[self.mm.market.syndicates[sy].syndicate_id] = None

        for i in range(len(self.mm.market.brokers)):
            self.mm.market.brokers[i].underwritten_contracts = []
            self.mm.market.brokers[i].not_underwritten_risks = []
            self.mm.market.brokers[i].not_paid_claims = []
        for i in range(len(self.mm.market.syndicates)):
            self.mm.market.syndicates[i].current_hold_contracts = []
            self.mm.market.syndicates[i].current_capital = self.syndicate_args["initial_capital"]
            self.mm.market.syndicates[i].current_capital_category = [self.syndicate_args["initial_capital"]/self.risk_args["num_categories"] for i in range(self.risk_args["num_categories"])]
        
        # Log data
        self.cumulative_bankruptcies = 0
        self.cumulative_market_exits = 0
        self.cumulative_unrecovered_claims = 0.0
        self.cumulative_claims = 0.0
        self.logger = logger.Logger(self.num_risk_models, self.catastrophes, self.brokers, self.syndicates)

        # Initiate time step
        self.timestep = -1
        self.step_track = 0

        return self.state_encoder_dict, info_dict
    
    def adjust_market_premium(self, capital):
        """Adjust_market_premium Method.
               Accepts arguments
                   capital: Type float. The total capital (cash) available in the insurance market (insurance only).
               No return value.
           This method adjusts the premium charged by insurance firms for the risks covered. The premium reduces linearly
           with the capital available in the insurance market and viceversa. The premium reduces until it reaches a minimum
           below which no insurer is willing to reduce further the price. """
        self.market_premium = self.fair_market_premium * (self.syndicate_args["upper_premium_limit"] 
                                                   - self.syndicate_args["premium_sensitivity"] 
                                                   * capital / (self.syndicate_args["initial_capital"] 
                                                   * self.risk_model_configs[0]["damage_distribution"].mean() * self.risk_args["num_risks"]))
        if self.market_premium < self.fair_market_premium * self.syndicate_args["lower_premium_limit"]:
            self.market_premium = self.fair_market_premium * self.syndicate_args["lower_premium_limit"]
    
    def get_actions(self):
        # Syndicates compete for the ledership, they will all cover 0.5
        sum_capital = sum([self.mm.market.syndicates[i].current_capital for i in range(len(self.mm.market.syndicates))]) 
        self.adjust_market_premium(capital=sum_capital)
        action_dict = {}
        for i in range(len(self.mm.market.syndicates)):
            action_dict.update({self.mm.market.syndicates[i].syndicate_id: self.market_premium})

        #{'0': array([0.9], dtype=float32), '1': array([0.67964387], dtype=float32), '2': array([0.77142656], dtype=float32)}

        return action_dict

    def step(self, action_dict):

        obs_dict, reward_dict, terminated_dict, info_dict = {}, {}, {}, {}
        flag_dict = {}

        # Update environemnt after actions
        parsed_actions = []        
        for syndicate_id, action in action_dict.items():
            # update action map
            self.action_map = self.action_map_creator(self.mm.market.syndicates[int(syndicate_id)],action)
            parsed_ac2add = self.action_map
            parsed_actions.append(parsed_ac2add)
        
        self.send_action2env(parsed_actions)

        # Evolve the market
        self.mm.evolve(self.dt)
        
        self.timestep += 1

        # Compute rewards and get next observation
        for syndicate_id, action in action_dict.items():
            reward_dict[syndicate_id] = self.compute_reward(action, syndicate_id)
            obs_dict[syndicate_id]= self.state_encoder(syndicate_id)
            info_dict[syndicate_id] = {}
            flag_dict[syndicate_id] = False
            terminated_dict[syndicate_id] = self.check_termination(syndicate_id)
            if terminated_dict[syndicate_id]:
                self.dones.add(syndicate_id)
        # Update plot 
        self.draw2file(self.mm.market)

        # All done termination check
        all_terminated = True
        for _, syndicate_terminated in terminated_dict.items():
            if syndicate_terminated is False:
                all_terminated = False
                break
        
        terminated_dict["__all__"] = all_terminated
        flag_dict["__all__"] = all_terminated

        return obs_dict, reward_dict, terminated_dict, flag_dict, info_dict

    def check_termination(self, syndicate_id):

        # Update per syndicate status, True-active in market, False-exit market becuase of no contract or bankruptcy
        market = self.mm.market
        sy = market.syndicates[int(syndicate_id)] 

        # The simulation is done when syndicates exit or bankrupt or reach the maximum time step
        if self.timestep >= self.maxstep-1:
            terminated = True
        else:
            terminated = False

        return terminated

    def compute_reward(self, action, syndicate_id):

        market = self.mm.market
        # calculate reward function
        r = [0.0] * 4

        # For each insurable risk being accepted +1 or refused -1
        if(self.timestep <= self.maxstep):
            for broker_id in range(len(market.brokers)):
                for risk in range(len(market.brokers[broker_id].risks)):
                    for contract in range(len(market.brokers[broker_id].underwritten_contracts)):
                        if market.brokers[broker_id].risks[risk]["risk_id"] == market.brokers[broker_id].underwritten_contracts[contract]["risk_id"]:
                            r[0] += 1
                        else:
                            r[0] -= 1

        # For each claim being paied +1 or refused -1
        if(self.timestep <= self.maxstep):
            for claim in range(len(market.syndicates[int(syndicate_id)].paid_claim)):
                if market.syndicate[syndicate_id].paid_claim[claim]["status"] == True:
                    r[1] += 1
                else:
                    r[1] -= 1

        # Profit and Bankruptcy       
        if(self.timestep <= self.maxstep):
            initial_capital = market.syndicates[int(syndicate_id)].initial_capital
            current_capital = market.syndicates[int(syndicate_id)].current_capital
            r[2] += current_capital - initial_capital
            if (current_capital - initial_capital) < 0:
                r[3] -= 10000

        # Sum reward
        reward = r[0] + r[1] + r[2] + r[3]

        return reward     

    def send_action2env(self, parsed_actions):               
            
        # Apply action
        if len(parsed_actions) > 0:
            self.mm.receive_actions(actions=parsed_actions) 
    
    def state_encoder(self, syndicate_id):
        
        ### Observation Space:             
        obs = []
        for risk in range(len(self.broker_risk_events)):
            if self.broker_risk_events[risk].risk_start_time == self.timestep+1:
                # Catastrophe risk category and risk value
                obs.append(self.broker_risk_events[risk].risk_category)
                obs.append(self.broker_risk_events[risk].risk_value)
        
        # Syndicates status current capital in 
        market = self.mm.market
        for num in range(len(market.syndicates[int(syndicate_id)].current_capital_category)):
            obs.append(market.syndicates[int(syndicate_id)].current_capital_category[num])
            
        return obs

    def action_map_creator(self, syndicate, line_size):

        action_map = None
        for risk in range(len(self.broker_risk_events)):
            if self.broker_risk_events[risk].risk_start_time == self.timestep+1:
                action_map = Action(syndicate.syndicate_id, line_size, self.broker_risk_events[risk].risk_id, self.broker_risk_events[risk].broker_id)
       
        return action_map
    
    def save_data(self):
        """Method to collect statistics about the current state of the simulation. Will pass these to the 
           Logger object (self.logger) to be recorded."""
        # Collect data
        total_cash = sum([self.mm.market.syndicates[i].current_capital for i in range(len(self.mm.market.syndicates))])
        total_excess_capital = sum([self.mm.market.syndicates[i].excess_capital for i in range(len(self.mm.market.syndicates))])
        total_profitslosses =  sum([self.mm.market.syndicates[i].profits_losses for i in range(len(self.mm.market.syndicates))])
        total_contracts = sum([len(self.mm.market.syndicates[i].current_hold_contracts) for i in range(len(self.mm.market.syndicates))])
        operational_syndicates = sum([self.mm.market.syndicates[i].status for i in range(len(self.mm.market.syndicates))])
        #operational_catbonds = sum([catbond.operational for catbond in self.catbonds])
        
        # Collect agent-level data
        syndicates_data = [(self.mm.market.syndicates[i].current_capital, 
                            self.mm.market.syndicates[i].syndicate_id, 
                            self.mm.market.syndicates[i].status) for i in range(len(self.mm.market.syndicates))]
        
        # Update cumulative information
        for i in range(len(self.mm.market.syndicates)):
            if self.mm.market.syndicates[i].status == False:
                self.cumulative_bankruptcies += 1
            if self.mm.market.syndicates[i].current_capital < self.mm.market.syndicates[i].capital_permanency_limit:         #If their level of cash is so low that they cannot underwrite anything they also leave the market.
                self.cumulative_market_exits += 1  # TODO: update the syndicates list becuase of market exit
            for j in range(len(self.mm.market.syndicates[i].current_hold_contracts)):
                if self.mm.market.syndicates[i].current_hold_contracts[j]["pay"] == False:
                    self.cumulative_unrecovered_claims += self.mm.market.syndicates[i].current_hold_contracts[j]["risk_value"]
                elif self.mm.market.syndicates[i].current_hold_contracts[j]["pay"] == True:
                    self.cumulative_claims += self.mm.market.syndicates[i].current_hold_contracts[j]["risk_value"]
        
        # Prepare dict
        current_log = {}
        current_log['total_cash'] = total_cash
        current_log['total_excess_capital'] = total_excess_capital
        current_log['total_profitslosses'] = total_profitslosses
        current_log['total_contracts'] = total_contracts
        current_log['total_operational'] = operational_syndicates
        #current_log['total_catbondsoperational'] = catbondsoperational_no
        current_log['market_premium'] = self.market_premium  # Oxford Picing has a fair premium and adjust TODO:  
        current_log['cumulative_bankruptcies'] = self.cumulative_bankruptcies
        current_log['cumulative_market_exits'] = self.cumulative_market_exits
        current_log['cumulative_unrecovered_claims'] = self.cumulative_unrecovered_claims
        current_log['cumulative_claims'] = self.cumulative_claims    #Log the cumulative claims received so far.
        
        # Add agent-level data to dict
        current_log['insurance_firms_cash'] = syndicates_data
        
        current_log['individual_contracts'] = []
        individual_contracts_no = [len(self.mm.market.syndicates[i].current_hold_contracts) for i in range(len(self.mm.market.syndicates))]
        for i in range(len(individual_contracts_no)):
            current_log['individual_contracts'].append(individual_contracts_no[i])

        # Call to Logger object
        self.logger.record_data(current_log)


In [6]:
### 6. Register environment and train the model

import gymnasium as gym
import numpy as np
import ray
from ray.tune.registry import register_env
from ray import air, tune
from ray.rllib.algorithms.ppo import PPO
from ipywidgets import IntProgress
from gym.spaces import Box
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.examples.policy.random_policy import RandomPolicy

insurance_args = {"sim_args": sim_args,
                "manager_args": manager_args,
                "broker_args": broker_args,
                "syndicate_args": syndicate_args,
                "reinsurancefirm_args": reinsurancefirm_args,
                "shareholder_args": shareholder_args,
                "risk_args": risk_args,
                "brokers": brokers,
                "syndicates": syndicates,
                "reinsurancefirms": reinsurancefirms,
                "shareholders": shareholders,
                "catastrophes": catastrophes,
                "broker_risks": broker_risks,
                "fair_market_premium": fair_market_premium,
                "risk_model_configs": risk_model_configs,
                "with_reinsurance": with_reinsurance,
                "num_risk_models": num_risk_models}

def env_creator(env_config):
    return MultiAgentBasedModel(**env_config)

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    # agent0 -> main0
    # agent1 -> main1
    return f"main{agent_id[-1]}"

def ppo_trainer_creator(insurance_args):
    
    config = {
        "env": "SpecialtyInsuranceMarket-validation",
        "framework": "tf",
        "multi_agent": {"policies":{
                # The Policy we are actually learning.
                "main0": PolicySpec(
                    observation_space=gym.spaces.Box(low=np.array([-10000000,-10000000,-30000000,-30000000,-30000000,-30000000]), 
                                                     high=np.array([10000000,10000000,30000000,30000000,30000000,30000000]), dtype = np.float32),
                    action_space=gym.spaces.Box(0.5, 0.9, dtype = np.float32)
                ),
                "main1": PolicySpec(
                    observation_space=gym.spaces.Box(low=np.array([-10000000,-10000000,-30000000,-30000000,-30000000,-30000000]), 
                                                     high=np.array([10000000,10000000,30000000,30000000,30000000,30000000]), dtype = np.float32),
                    action_space=gym.spaces.Box(0.5, 0.9, dtype = np.float32)
                ),
                "random": PolicySpec(policy_class=RandomPolicy),
            }, 
                        "policy_mapping_fn": policy_mapping_fn,
                        "policies_to_train":["main0"],
        },
        "observation_space": gym.spaces.Box(low=np.array([-10000000,-10000000,-30000000,-30000000,-30000000,-30000000]), 
                                            high=np.array([10000000,10000000,30000000,30000000,30000000,30000000]), dtype = np.float32),
        "action_space": gym.spaces.Box(0.5, 0.9, dtype = np.float32),
        "env_config": insurance_args,
        "evaluation_interval": 2,
        "evaluation_duration": 20,
    }
    
    trainer = PPO(config=config)
    return trainer

# Folder for recording
top_dir = "noreinsurance_" + "_model_" + str(num_risk_models)

# Register environment
register_env("SpecialtyInsuranceMarket-validation", env_creator)

# The number of training iteration for the RL agent
num_training = 10

trainer = ppo_trainer_creator(insurance_args)
# Number of training iterations

"""
for n in range(num_training):
    # Create a path to store the trained agent for each iteration
    model_filepath = f"{top_dir}/{str(n)}/saved_models"
        
    num_episode = 10

    # A training iteration includes parallel sample collection by the environment workers 
    # as well as loss calculation on the collected batch and a model update.

    bar = IntProgress(min=0, max=num_episode)
    display(bar)
    list_mean_rewards = []
    list_min_rewards = []
    list_max_rewards = []
    list_train_step = []

    for i in range(num_episode):
        trainer.train()     
        print("Progress:", i+1, "/", num_episode, end="\r")
        bar.value += 1
        if (i+1) % 2 == 0:
            list_mean_rewards.append(trainer.evaluation_metrics["evaluation"]["episode_reward_mean"])
            list_min_rewards.append(trainer.evaluation_metrics["evaluation"]["episode_reward_min"])
            list_max_rewards.append(trainer.evaluation_metrics["evaluation"]["episode_reward_max"])
            list_train_step.append(i+1)
        if i % 10 == 0:
            trainer.save(model_filepath)
"""  
    

# Can be used for game model
env = MultiAgentBasedModel(**insurance_args)
    
total_steps = 0
terminated_dict = {"__all__": False}
    
obs_dict, info_dict = env.reset()

while not terminated_dict["__all__"]:
    if total_steps % 20 == 0: print(".", end="")
        
    action_dict = env.get_actions()
    print(action_dict)
    total_steps += 1
    
    obs_dict, reward_dict, terminated_dict, flag_dict, info_dict = env.step(action_dict)


/Users/k2472543/Library/Python/3.8/lib/python/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/Users/k2472543/Library/Python/3.8/lib/python/site-packages/ray/rllib/algorithms/algorithm.py:483: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/k2472543/Library/Python/3.8/lib/python/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface

.{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
.{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.07038, '1': 0.07038, '2': 0.07038}
{'0': 0.0

In [ ]:

### 7. Test the trained model performance

def trainer_restore(self, top_dir, n):
    if n <= 9:
        path0 = top_dir
        path1 = str(n-1)
        path2 = "saved_models"
        path3 = "checkpoint_"+str(0)+str(0)+str(0)+str(0)+str(0)+str(n)
        path4 = "rllib_checkpoint.json"
    elif 9 < n <= 99:
        path0 = top_dir
        path1 = str(n-1)
        path2 = "saved_models"
        path3 = "checkpoint_"+str(0)+str(0)+str(0)+str(0)+str(n)
        path4 = "rllib_checkpoint.json"
    elif 99 < n <= 999:
        path0 = top_dir
        path1 = str(n-1)
        path2 = "saved_models"
        path3 = "checkpoint_"+str(0)+str(0)+str(0)+str(n)
        path4 = "rllib_checkpoint.json"

    # Join various path components
    self.trainer.restore(os.path.join(path0, path1, path2, path3, path4))

insurance_args = {"sim_args": sim_args,
                "manager_args": manager_args,
                "broker_args": broker_args,
                "syndicate_args": syndicate_args,
                "reinsurancefirm_args": reinsurancefirm_args,
                "shareholder_args": shareholder_args,
                "risk_args": risk_args,
                "brokers": brokers,
                "syndicates": syndicates,
                "reinsurancefirms": reinsurancefirms,
                "shareholders": shareholders,
                "catastrophes": catastrophes,
                "broker_risks": broker_risks,
                "fair_market_premium": fair_market_premium,
                "risk_model_configs": risk_model_configs,
                "with_reinsurance": with_reinsurance,
                "num_risk_models": num_risk_models}

validation_episodes = 1
all_rewards = {}
        
for epi in range(validation_episodes):
    env = MultiAgentBasedModel(**insurance_args)
    
    print(f"\nepisode: {epi} | ")
    total_steps = 0
    terminated_dict = {"__all__": False}
    all_rewards[epi] = {}
    
    obs_dict, info_dict = env.reset()
    
    while not terminated_dict["__all__"]:
        if total_steps % 20 == 0: print(".", end="")
        
        action_dict = trainer.compute_actions(obs_dict)  
        total_steps += 1
        
        obs_dict, reward_dict, terminated_dict, flag_dict, info_dict = env.step(action_dict)
        print(total_steps)
        print(action_dict)
        for k, v in reward_dict.items():
            if k not in all_rewards[epi]:
                all_rewards[epi][k] = [v]
            else:
                all_rewards[epi][k].append(v)




In [ ]:
### Main function run the simulation, two syndicates will be chosen to compete for the leader position
from manager.ai_model.runner import AIRunner
from manager.game_model.runner import GameRunner

model = 0
if model == 0: 
    runner = AIRunner(sim_args, manager_args, brokers, syndicates, reinsurancefirms, shareholders, catastrophes, risk_model_configs, with_reinsurance, num_risk_models)
elif model == 1:
    runner = GameRunner(sim_args, manager_args, brokers, syndicates, reinsurancefirms, shareholders, catastrophes, risk_model_configs, with_reinsurance, num_risk_models)
runner.run()
